Spusteni pyspark

`pyspark --master yarn --num-executors 2 --executor-memory 4G --packages com.databricks:spark-csv_2.10:1.5.0 --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je vas den a mesic narozeni, napr. spark.ui.port=10811

In [ ]:
# uzitecne importy

from pyspark.sql.types import *
from pyspark.sql import functions as F

[Manual Spark SQL](http://spark.apache.org/docs/1.6.0/api/python/pyspark.sql.html)

### 1. Nacteni dat a zakladni explorace

Soubor lyrics.txt obsahuje zaznamy o pisnich vcetne textu. Tento soubor nactete jako DataFrame a pri nacteni definujte schema -- viz nize.

* format: CSV
* cesta `/user/pascepet/data/`
* oddelovac je `','`
* schema
```
id – long
album – string
rok – integer
interpret – string
zanr – string
text – string
```


In [ ]:
# nejprve definice schematu
schema_lyrics = StructType([
    StructField("id", LongType(), True),
    StructField("nazev", StringType(), True),
    StructField("rok", IntegerType(), True),
    StructField("interpret", StringType(), True),
    StructField("zanr", StringType(), True),
    StructField("text", StringType(), True)])

# nacteni souboru
songsDF = sqlContext.read \
    .format("com.databricks.spark.csv") \
    .option("header", "false") \
    .option("delimiter", ",") \
    .schema(schema_lyrics)  \
    .load("/user/pascepet/data/lyrics.txt")


1.1 Nactene DataFrame nakesujte.    
`songsDF.cache()`

1.2 Vypiste si ukazku dat.    
`songsDF.show()`

1.3 Zjistete pocet zaznamu (radku) v DataFrame.    
`songsDF.count()`

1.4 Zjistete, kolik pisni ma jako interpreta Boba Dylana ('bob-dylan').    
`songsDF.filter(songsDF.interpret=='bob-dylan').count()`

1.5 Zjistete, jaky je nejnizsi a nejvyssi uvedeny rok pisne.    
`songsDF.groupBy().min('rok').show()
songsDF.groupBy().max('rok').show()`

1.6 Zjistete, ktery zanr ma nejvic pisni.    
`songs_zanry = songsDF.select('zanr').groupBy('zanr').count().toDF('zanr', 'pocet')
songs_zanry.orderBy(songs_zanry.pocet.desc()).show()`    
`# nebo songs_zanry.orderBy(F.desc('pocet')).show()`


### 2. Uprava a cisteni dat

2.1 Vyradte vsechny zaznamy, ktere maji uvedeny rok mimo rozmezi 1950--2018. Zjistete, kolik zaznamu v DataFrame zustalo.

`songsDF = songsDF.filter('rok>=1950 and rok<=2018')`    
`# nebo songsDF.filter((songsDF.rok>=1950) & (songsDF.rok<=2018))`    
`songsDF.count()`

2.2 Z textu pisne odstrante vsechny znaky ',.:;!?()[]' (obtiznejsi varianta: odstrante vsechny nealfanumericke znaky krome mezer) a text prevedte na mala pismena.

`songsDF = songsDF.withColumn('text', F.regexp_replace(songsDF.text, r'[,.;:?!()\[\]]', ''))`    
`# obtiznejsi varianta: songsDF = songsDF.withColumn('text', F.regexp_replace(songsDF.text, '[^\w ]', ''))`    
`songsDF = songsDF.withColumn('text', F.lower(songsDF.text))`

2.3 Pridejte sloupec `slova_poc` obsahujici pocet vsech slov pisne, sloupec `slova_poc_unik` obsahujici pocet vsech unikatnich slov pisne a sloupec `slova_poc_unik2` obsahujici pocet vsech unikatnich slov pisne po vyrazeni stop-words. Slova jsou v textu oddelena carkou. Soubor se stop-words je na HDFS: `/user/pascepet/data/stopwords.txt`.

`# pocet vsech slov lze zjistit rovnou - rozdelit na slova a spocitat prvky pole
songsDF = songsDF.withColumn('slova_poc', F.size(F.split(songsDF.text, ' ')))`

`# pro spocteni unikatnich slov je ale potreba mit sloupec s rozdelenym textem jako mezikrok`    
`# rozdelit na slova do noveho sloupce => v poli je array jednotlivych slov
songsDF = songsDF.withColumn('text_upr', F.split(songsDF.text, ' '))`    
`# pocet unikatnich - abychom mohli pouzit funkce Pythonu, musime definovat uzivatelskou funkci
countUniq = F.udf(lambda a: len(set(a)))`    
`# ... a tu aplikovat na array ve sloupci
songsDF = songsDF.withColumn('slova_poc_unik', countUniq(songsDF.text_upr))`

`# pro spocteni unikatnich slov bez stop-words jste nacteme soubor se stop-words
stopw = sc.textFile("/user/pascepet/data/stopwords.txt").collect()
stopw = set(stopw)`    
`# ... a postupujeme stejne jako vyse
countUniqNotStop = F.udf(lambda a: len(set(a).difference(stopw)))
songsDF = songsDF.withColumn('slova_poc_unik2', countUniqNotStop(songsDF.text_upr))`

`songsDF = songsDF.drop('text_upr') # vyhodime sloupec s upravenym textem`


2.4 Vysledny DataFrame opet nakesujte.    
`songsDF.cache()`









### 3. Analyticke dotazy

3.1 Zjistete pocty pisni podle jednotlivych roku.    
`songsDF.groupBy('rok').count().show()`

3.2 Zjistete, kolik interpretu ma aspon 500 pisni a kteri to jsou.    
`interprets = songsDF.groupBy('interpret').count() \
  .toDF('interpret', 'pocet').filter("pocet >= 500")
interprets.show()`

3.3 Ktery interpret ma v prumeru nejvice unikatnich slov na jednu pisen? A zmeni se to, pokud se budou pocitat jen unikatni slova bez stop-words? (Obtiznejsi varianta: berte v uvahu jen interprety, kteri maji aspon 50 pisni.)

`# ukazka prace s DataFrame pomoci SQL dotazu 
songsDF.registerTempTable("songs")
slova_prum = sqlContext.sql("""select interpret, avg(slova_poc_unik) as slova_unik_prum, avg(slova_poc_unik2) as slova_unik2_prum, count(*) as pisne_pocet from songs
    group by interpret""")`
    
`slova_prum.select('interpret', 'slova_unik_prum').orderBy(F.desc('slova_unik_prum')).show()
slova_prum.select('interpret', 'slova_unik2_prum').orderBy(F.desc('slova_unik2_prum')).show()
slova_prum.filter('pisne_pocet>=50').orderBy(F.desc('slova_unik_prum')).show()
slova_prum.filter('pisne_pocet>=50').orderBy(F.desc('slova_unik2_prum')).show()`

### 4. Text-mining

4.1 Najdete 20 nejcasteji se vyskytujicich slov bez stop-words. (Kazde slovo pocitejte tolikrat, kolikrat je v textu uvedeno.)    
`# texty rozdelit na slova`    
`# pustit dal jen slova, ktera nepatri do stopwords (stopw je drive nacteny soubor, typ set)`    
`# a klasicky word-count s tridenim podle poctu`    
`words_top = songsDF.flatMap(lambda r: r[5].split(" ")) \
    .filter(lambda r: r not in stopw) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda r: r[1], False)`
    
`words_top.take(20)`

4.2 Vyberte 3 nejcastejsi slova (krome stop-words) a k DataFrame pridejte tri sloupce s priznaky, zda je v pisni dane slovo aspon jednou uvedeno.    
`# priklad pro slova love, like, know
songsDF = songsDF.withColumn('is_love', F.when(F.regexp_extract(songsDF.text, r'\b(love)\b', 1) == 'love', 1).otherwise(0))
songsDF = songsDF.withColumn('is_like', F.when(F.regexp_extract(songsDF.text, r'\b(like)\b', 1) == 'like', 1).otherwise(0))
songsDF = songsDF.withColumn('is_know', F.when(F.regexp_extract(songsDF.text, r'\b(know)\b', 1) == 'know', 1).otherwise(0))`

4.3 U interpretu s aspon 500 pisnemi (viz 3.2) zjistete, v jakem podilu jejich pisni se vyskytuji tri vami vybrana nejcastejsi slova.    
`interprets_words = interprets.join(songsDF, interprets.interpret==songsDF.interpret) \
    .drop(songsDF.interpret) \
    .select('interpret', 'is_love', 'is_like', 'is_know') \
    .groupBy('interpret') \
    .agg({'is_love':'avg', 'is_like':'avg', 'is_know':'avg'})`

`interprets_words.show()`
